In [1]:
import numpy as np
import pandas as pd

In [2]:
hea_20 = pd.read_stata('ch20m_EN_1.0p.dta')
rel_20 = pd.read_stata('cr20m_EN_1.1p.dta')
soc_20 = pd.read_stata('cs20m_EN_1.1p.dta')
fam_20 = pd.read_stata('cf20m_EN_1.0p.dta')
wor_20 = pd.read_stata('cw20m_EN_1.0p.dta')
per_20 = pd.read_stata('cp20l_EN_1.0p.dta')
val_20 = pd.read_stata('cv20l_EN_1.0p.dta')
ass_20 = pd.read_stata('ca20g_EN_1.0p.dta')
inc_20 = pd.read_stata('ci20m_EN_1.0p.dta')
hou_20 = pd.read_stata('cd20m_EN_2.0p.dta')

In [3]:
all_20 = [ass_20, hou_20, fam_20, hea_20, inc_20, per_20, soc_20, val_20, wor_20]

merged_20 = all_20[0]

for all in all_20[1:]:
    merged_20 = pd.merge(merged_20, all, on='nomem_encr', how='inner')

In [4]:
merged_20.head()

,nomem_encr,ca20g_m,ca20g001,ca20g002,ca20g003,ca20g082,ca20g078,ca20g005,ca20g006,ca20g007,...,cw20m496,cw20m497,cw20m498,cw20m499,cw20m500,cw20m501,cw20m502,cw20m503,cw20m504,cw20m505
0,800015.0,202009.0,yes,yes,head of household*,57.0,yes,no,no,no,...,1 = Certainly not,5 = Certainly yes,1 = Certainly not,1 = Certainly not,1 = Certainly not,21-04-2020,21:40:12,21-04-2020,21:56:56,1004.0
1,800170.0,202009.0,no,yes,head of household*,61.0,yes,no,no,no,...,1 = Certainly not,5 = Certainly yes,3.0,5 = Certainly yes,5 = Certainly yes,14-04-2020,21:53:39,14-04-2020,22:11:20,1061.0
2,800186.0,202009.0,no,yes,head of household*,81.0,yes,no,no,no,...,2.0,4.0,4.0,4.0,4.0,13-04-2020,15:42:20,13-04-2020,15:49:15,415.0
3,800201.0,202009.0,yes,yes,married partner,47.0,yes,no,no,no,...,1 = Certainly not,5 = Certainly yes,2.0,2.0,1 = Certainly not,13-04-2020,12:37:58,13-04-2020,12:50:42,764.0
4,800204.0,202009.0,yes,no,head of household*,78.0,yes,no,no,no,...,1 = Certainly not,5 = Certainly yes,5 = Certainly yes,3.0,3.0,15-04-2020,17:33:02,15-04-2020,17:41:04,482.0
